In [45]:
import sys
parent_dir='C:\\Users\\garav\\AGOP\\DLR'
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [54]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import trainer as t
from utils import agop_conv as agc
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
import rfm
import numpy as np
import visdom
import model1
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm
from torchvision import models


In [56]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f"Using device: {device}")

Using device: cuda:0


In [7]:
pwd

'C:\\Users\\garav\\AGOP\\rfm_mnist'

In [58]:
def one_hot_data(dataset, num_samples=-1):
    labelset = {}
    for i in range(10):
        one_hot = torch.zeros(10)
        one_hot[i] = 1
        labelset[i] = one_hot

    subset = [(ex.flatten(), labelset[label]) for \
              idx, (ex, label) in enumerate(dataset) if idx < num_samples]
    return subset


def group_by_class(dataset):
    labelset = {}
    for i in range(10):
        labelset[i] = []
    for i, batch in enumerate(dataset):
        img, label = batch
        labelset[label].append(img.view(1, 3, 32, 32))
    return labelset


def split(trainset, p=.8):
    train, val = train_test_split(trainset, train_size=p)
    return train, val

def merge_data(mnist, n):
    #cifar_by_label = group_by_class(cifar)

    mnist_by_label = group_by_class(mnist)

    data = []
    labels = []

    labelset = {}

    for i in range(10):
        one_hot = torch.zeros(1, 10)
        one_hot[0, i] = 1
        labelset[i] = one_hot

    for l in mnist_by_label:

        #cifar_data = torch.cat(cifar_by_label[l])
        mnist_data = torch.cat(mnist_by_label[l])
        min_len = len(mnist_data)
        m = min(n, min_len)
        #cifar_data = cifar_data[:m]
        mnist_data = mnist_data[:m]

        merged = torch.cat([mnist_data], axis=-1)
        #for i in range(3):
           # vis.image(merged[i])
        data.append(merged.reshape(m, -1))
        print(merged.shape)
        labels.append(np.repeat(labelset[l], m, axis=0))
    data = torch.cat(data, axis=0)

    labels = np.concatenate(labels, axis=0)
    merged_labels = torch.from_numpy(labels)

    return list(zip(data, labels))



In [60]:
torch.cuda.empty_cache()

In [ ]:

SEED = 5700
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
#cudnn.benchmark = False

transform = transforms.Compose(
        [transforms.ToTensor()
        ])

def repeat_channel(x):
    return x.repeat(3, 1, 1)

mnist_transform = transforms.Compose(
    [transforms.Resize([32, 32]),
     transforms.ToTensor(),
     transforms.Lambda(repeat_channel)]
)

path= './data'  
    
mnist_trainset = torchvision.datasets.MNIST(root=path,
                                                train=True,
                                                transform=mnist_transform,
                                                download=True)

#trainset = group_by_class(mnist_trainset)
trainset = merge_data(mnist_trainset, 5000)
trainset, valset = split(trainset, p=.8)
print("Train Size: ", len(trainset), "Val Size: ", len(valset))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                              shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=100,
                                            shuffle=False, num_workers=1)


mnist_testset = torchvision.datasets.MNIST(root=path,
                                               train=False,
                                               transform=mnist_transform,
                                               download=True)

print("Test Size: ", len(mnist_testset))
testset = merge_data(mnist_testset, 900)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                             shuffle=False, num_workers=2)

name = 'mnist'


In [27]:
for idx, batch in enumerate(trainloader):
        inputs, labels = batch
        _, dim = inputs.shape
        break
print(dim)

3072


In [33]:
net = model1.Net(3072, num_classes=10)

In [ ]:
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)
#initialize_weights(model)
import os
if os.path.exists('nn_models/mnist_trained_nn.pth'):
    checkpoint = torch.load('nn_models/mnist_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")

t.train_network(trainloader, valloader, testloader,
                num_classes=10, optimizer= torch.optim.SGD(net.parameters(), lr=.1)
                name=name, net=net)

In [ ]:
'''Warning: This is an extremely cpu intensive process since it uses solve function from linalg 
The rfm.py from utils is equipped with more memory efficient solvers. 
'''

rfm.rfm(trainloader, valloader, testloader, name=name,
            batch_size=10, iters=1, reg=1e-3)